In [1]:
import os
import PyPDF2 as pdf
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI  # <-- Corrected import

os.environ['OPENAI_API_KEY']= "sk-proj-6BWDTDaGs4ulJSkT9S2sT3BlbkFJYKsskXdic6VLfHsDpvFJ"

In [2]:
def getTextFromPDF()->str:
    with open('Quiz.pdf',mode='rb') as contentFile:
        convertedText = ""
        pdf_reader = pdf.PdfReader(contentFile)
        for i in range(0,len(pdf_reader.pages)):
            page = pdf_reader.pages[i]
            pageText = page.extract_text()
            convertedText+=pageText
        file_path = "Loader.txt"
        with open(file_path, "w+") as file:
            file.write(convertedText)
            file.seek(0)
        return file_path

In [4]:
loader = TextLoader(getTextFromPDF())
documents = loader.load()
text_splitter  =  RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=50)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
store = Chroma.from_documents(texts,embeddings,collection_name="Quiz-PDF")

In [5]:
llm = ChatOpenAI(temperature=0.8, model="gpt-3.5-turbo")
chain = RetrievalQA.from_chain_type(llm,retriever=store.as_retriever())

In [10]:
prompt = "The document consists of questions and multiple choice options, I want you to return a response in JSON format, the fomat contains question_number, question_text, Multiple choice questions and answer, for each question that you find in a document return a separate JSON in the provided format"
response = chain.run(prompt)

In [11]:
print(response)

```json
{
    "question_number": 1,
    "question_text": "Major manufactured food product is ………………..",
    "multiple_choice": {
        "a": "Sugar",
        "b": " ",
        "c": " ",
        "d": " ",
        "e": " "
    },
    "answer": "a"
}
```

```json
{
    "question_number": 2,
    "question_text": "Quality Factors in Food industry ……………………..",
    "multiple_choice": {
        "a": " ",
        "b": "Appearance Factors",
        "c": " ",
        "d": " ",
        "e": " "
    },
    "answer": "b"
}
```

```json
{
    "question_number": 3,
    "question_text": "………………..usually measured by counts of bacteria, yeast, mold, and insect",
    "multiple_choice": {
        "a": " ",
        "b": " ",
        "c": " ",
        "d": " ",
        "e": "Sanitary Quality"
    },
    "answer": "e"
}
```

```json
{
    "question_number": 4,
    "question_text": "Water effluent generated in printing industry is decolourised by………………….",
    "multiple_choice": {
        "a": "Electrolytic d